# Credit Risk Resampling Techniques

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import datetime

# Read the CSV and Perform Basic Data Cleaning

In [11]:
columns = ["loan_amnt", "int_rate", "annual_inc", "open_acc", "total_acc", "total_pymnt", "loan_status"]

target = ["loan_status"]

In [12]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,annual_inc,open_acc,total_acc,total_pymnt,loan_status
0,35000.0,0.1308,125890.0,14.0,29.0,35016.528333,Fully Paid
1,20000.0,0.2250,75000.0,8.0,11.0,719.000000,low_risk
2,10500.0,0.1719,66000.0,8.0,14.0,355.290000,low_risk
3,25000.0,0.2000,105000.0,17.0,34.0,873.530000,low_risk
4,20000.0,0.2000,56000.0,8.0,21.0,485.440000,low_risk


# Split the Data into Training and Testing

In [13]:
# Create our features
# BCS SAYS TO DELETE THE CELL ABOVE AND ADD THE FOLL
X = df.drop(columns="loan_status")


# Create our target
y = df["loan_status"]

In [14]:
X.describe()

,loan_amnt,int_rate,annual_inc,open_acc,total_acc,total_pymnt
count,96840.000000,96840.000000,9.684000e+04,96840.000000,96840.000000,96840.000000
mean,16642.454822,0.126793,8.516747e+04,11.707239,23.079864,1272.141466
std,10365.336053,0.048470,1.133767e+05,5.978739,12.186639,2796.006935
min,1000.000000,0.060000,0.000000e+00,0.000000,2.000000,0.000000
25%,9000.000000,0.081900,4.900000e+04,7.000000,14.000000,408.060000
50%,15000.000000,0.118000,7.000000e+04,11.000000,21.000000,726.530000
75%,24000.000000,0.155700,1.000000e+05,15.000000,30.000000,1258.670000
max,40000.000000,0.308900,9.000000e+06,72.000000,164.000000,41418.981165


In [15]:
# Check the balance of our target values
y.value_counts()

low_risk       94116
Fully Paid      2157
high_risk        526
Charged Off       41
Name: loan_status, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 70569,
         'Fully Paid': 1644,
         'high_risk': 385,
         'Charged Off': 32})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [17]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 70569,
         'Fully Paid': 70569,
         'high_risk': 70569,
         'Charged Off': 70569})

In [18]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

NameError: name 'y_pred' is not defined

In [20]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[    9,     0,     0,     0],
       [    0,   513,     0,     0],
       [   53,     0,    49,    39],
       [  418,    39, 10147, 12943]], dtype=int64)

In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.02      1.00      0.98      0.04      0.99      0.98         9
 Fully Paid       0.93      1.00      1.00      0.96      1.00      1.00       513
  high_risk       0.00      0.35      0.58      0.01      0.45      0.20       141
   low_risk       1.00      0.55      0.94      0.71      0.72      0.50     23547

avg / total       0.99      0.56      0.94      0.71      0.72      0.51     24210



### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'low_risk': 70569,
         'Fully Paid': 70569,
         'high_risk': 70569,
         'Charged Off': 70569})

In [23]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7342018091476621

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[    9,     0,     0,     0],
       [    0,   513,     0,     0],
       [   55,     0,    46,    40],
       [  233,    39,  8898, 14377]], dtype=int64)

In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.03      1.00      0.99      0.06      0.99      0.99         9
 Fully Paid       0.93      1.00      1.00      0.96      1.00      1.00       513
  high_risk       0.01      0.33      0.63      0.01      0.45      0.20       141
   low_risk       1.00      0.61      0.94      0.76      0.76      0.55     23547

avg / total       0.99      0.62      0.94      0.76      0.76      0.56     24210



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [27]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Charged Off': 32, 'Fully Paid': 32, 'high_risk': 32, 'low_risk': 32})

In [28]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7342018091476621

In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[    8,     0,     1,     0],
       [    1,   490,     0,    22],
       [   44,     0,    53,    44],
       [  452,   123,  7184, 15788]], dtype=int64)

In [31]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.02      0.89      0.98      0.03      0.93      0.86         9
 Fully Paid       0.80      0.96      0.99      0.87      0.97      0.95       513
  high_risk       0.01      0.38      0.70      0.01      0.51      0.26       141
   low_risk       1.00      0.67      0.90      0.80      0.78      0.59     23547

avg / total       0.99      0.67      0.90      0.80      0.78      0.60     24210



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [32]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'Charged Off': 91936,
         'Fully Paid': 93854,
         'high_risk': 86453,
         'low_risk': 84461})

In [33]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7226074788335486

In [35]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[    9,     0,     0,     0],
       [    0,   512,     1,     0],
       [   50,     0,    55,    36],
       [  254,    44,  9634, 13615]], dtype=int64)

In [36]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.03      1.00      0.99      0.06      0.99      0.99         9
 Fully Paid       0.92      1.00      1.00      0.96      1.00      1.00       513
  high_risk       0.01      0.39      0.60      0.01      0.48      0.23       141
   low_risk       1.00      0.58      0.95      0.73      0.74      0.53     23547

avg / total       0.99      0.59      0.94      0.73      0.74      0.54     24210

